# 유사도 챗봇 성능 향상 기법
🤔 키워드는 잡아내면서 질문의 맥락도 캐치하고 싶다<br>
💡 BM25 + Faiss 앙상블 모델을 결합하여 유사도를 측정해 보자!

### 데이터 불러와서 리스트 생성

In [11]:
!pip install langchain langchain_community kiwipiepy rank_bm25 sentence_transformers faiss-gpu datasets transformers

In [1]:
import pandas as pd

# Excel 파일 경로 지정
file_path = '중학사회(취합).xlsx'

# Excel 파일 읽기
df = pd.read_excel(file_path, engine='openpyxl')

# 질문과 답변 리스트 생성
questions = df['SENT_CONTENT'].tolist()
int_sentences = df['INT_SENTENCE'].tolist()
int_responses = df['INT_RESPONSE'].tolist()

print("총 질문 개수:", len(questions))

총 질문 개수: 4867


# 1. BM25 단독

In [5]:
from rank_bm25 import BM25Okapi
from langchain_community.retrievers import BM25Retriever
from kiwipiepy import Kiwi
from langchain.schema import Document

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_questions = [kiwi_tokenize(question) for question in questions]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_questions)

# 사용자 질문 입력
query = "우리나라 정부 형태가 뭐야?"

documents = [Document(page_content=question, metadata={'index': i}) for i, question in enumerate(questions)]

# BM25 모델 설정
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

retriever = kiwi_bm25

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']

    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 총재 정부란 무엇인가요?
답변: 총재 정부는 프랑스 혁명 당시 로베스피에르의 가혹한 공포 정치에 반발하여 일어난 정부로 5명의 총재가 행정을 담당하였습니다.

질문: 우리나라 대통령제에서의 의원내각제적 요소는 무엇인가요?
답변: 1. 국무총리제, 2. 국회의원의 장관직 겸직 가능, 3. 행정부의 법률안 제출권입니다.



# 2. Faiss 단독

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

import torch
import pandas as pd
import numpy as np

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.schema import Document

from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_questions = [kiwi_tokenize(question) for question in questions]

# 사용자 질문 입력
query = "우리나라 정부 형태가 뭐야?"

documents = [Document(page_content=question, metadata={'index': i}) for i, question in enumerate(questions)]

# FAISS 설정
hf_model_name = "intfloat/multilingual-e5-large"  # contextualized embedding model 사용
hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
faiss = FAISS.from_documents(documents, hf_embeddings).as_retriever()

retriever = faiss

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']

    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

<ipython-input-3-59868297845c>:33: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 우리나라 대통령제에서의 의원내각제적 요소는 무엇인가요?
답변: 1. 국무총리제, 2. 국회의원의 장관직 겸직 가능, 3. 행정부의 법률안 제출권입니다.



# 3. 앙상블 모델 (1)

### 사용자 쿼리-데이터셋 질문 유사도 검색

In [7]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

import torch
import pandas as pd
import numpy as np

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.schema import Document

from kiwipiepy import Kiwi

file_path = '중학사회(취합).xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

# 질문과 답변 리스트 생성
questions = df['SENT_CONTENT'].tolist()
int_sentences = df['INT_SENTENCE'].tolist()
int_responses = df['INT_RESPONSE'].tolist()

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_questions = [kiwi_tokenize(question) for question in questions]

documents = [Document(page_content=question, metadata={'index': i}) for i, question in enumerate(questions)]

# BM25 모델 설정
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

# FAISS 설정
hf_model_name = "intfloat/multilingual-e5-large"  # contextualized embedding model 사용
hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
faiss = FAISS.from_documents(documents, hf_embeddings).as_retriever()

# EnsembleRetriever에서 검색 수행
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [8]:
query = "우리나라 정부 형태가 뭐야?"

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']
    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 우리나라 대통령제에서의 의원내각제적 요소는 무엇인가요?
답변: 1. 국무총리제, 2. 국회의원의 장관직 겸직 가능, 3. 행정부의 법률안 제출권입니다.

질문: 총재 정부란 무엇인가요?
답변: 총재 정부는 프랑스 혁명 당시 로베스피에르의 가혹한 공포 정치에 반발하여 일어난 정부로 5명의 총재가 행정을 담당하였습니다.



### 사용자 쿼리-데이터셋 답변 유사도 검색

In [9]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

import torch
import pandas as pd
import numpy as np

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.schema import Document

from kiwipiepy import Kiwi

file_path = '중학사회(취합).xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

# 질문과 답변 리스트 생성
questions = df['SENT_CONTENT'].tolist()
int_sentences = df['INT_SENTENCE'].tolist()
int_responses = df['INT_RESPONSE'].tolist()

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_int_responses = [kiwi_tokenize(response) for response in int_responses]

documents = [Document(page_content=response, metadata={'index': i}) for i, response in enumerate(int_responses)]

# BM25 모델 설정
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

# FAISS 설정
hf_model_name = "intfloat/multilingual-e5-large"  # contextualized embedding model 사용
hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
faiss = FAISS.from_documents(documents, hf_embeddings).as_retriever()

# EnsembleRetriever에서 검색 수행
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [10]:
query = "우리나라 정부 형태가 뭐야?"

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']
    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 대통령제와 의원 내각제의 차이점이 무엇인가요?
답변: 대통령제는 입법부와 행정부가 엄격하게 분리된 정부 형태입니다.
의원 내각제는 입법부와 행정부의 권한이 융합된 정부 형태입니다.
대통령제와 의원 내각제를 구분하는 방법은 정부와 의회의 융합 정도를 보면 됩니다.
정부와 의회가 철저하게 구분되어 있으면 대통령제, 정부와 의회가 융합되어 있으면 의원 내각제입니다.



# 4. 앙상블 모델 (2)

### 데이터셋의 답변 내용에 따른 카테고리 추가

In [12]:
import pandas as pd
from datasets import Dataset
from transformers import pipeline

# 파이프라인 설정 (GPU 사용)
pipe = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", device=0)

# JSON 파일 경로 지정
file_path = '중학사회(취합).xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 카테고리 후보들
candidate_labels = ["정치", "경제", "사회", "문화"]

# 배치 처리로 카테고리 할당
def categorize_batch(examples):
    result = pipe(examples['INT_RESPONSE'], candidate_labels)
    return {"CATEGORY": [res['labels'][0] for res in result]}  # 각 입력에 대해 가장 높은 점수의 카테고리 선택

# dataset에 파이프라인 적용
dataset = dataset.map(categorize_batch, batched=True)

model.safetensors:  26%|##6       | 147M/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4867 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# 결과를 DataFrame으로 변환 후 엑셀로 저장
df_result = dataset.to_pandas()
df_result.to_excel('./중학사회(취합)_카테고리 추가.xlsx', index=False, engine='openpyxl')

### 사용자 쿼리와 동일한 카테고리의 답변을 가진 질의응답 대상으로 검색 수행

### 사용자 쿼리-데이터셋 질문 유사도 검색

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

import torch
import pandas as pd
import numpy as np

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.schema import Document

from kiwipiepy import Kiwi

file_path = '중학사회(취합)_카테고리 추가.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

# 질문과 답변 리스트 생성
questions = df['SENT_CONTENT'].tolist()
int_sentences = df['INT_SENTENCE'].tolist()
int_responses = df['INT_RESPONSE'].tolist()

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_questions = [kiwi_tokenize(question) for question in questions]

# 사용자 질문에 대한 카테고리 결정
query = "우리나라 정부 형태가 뭐야?"
category = '정치'

# 특정 카테고리에 해당하는 문서만 필터링
filtered_documents = []
for i, question in enumerate(questions):
    doc_category = df.loc[i, 'CATEGORY']
    if doc_category == category:
        filtered_documents.append(Document(page_content=question, metadata={'index': i, 'category': doc_category}))

# BM25 모델 설정
kiwi_bm25 = BM25Retriever.from_documents(filtered_documents, preprocess_func=kiwi_tokenize)

# FAISS 설정
hf_model_name = "intfloat/multilingual-e5-large"  # contextualized embedding model 사용
hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
faiss = FAISS.from_documents(filtered_documents, hf_embeddings).as_retriever()

In [14]:
# EnsembleRetriever에서 검색 수행
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']
    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 우리나라 대통령제에서의 의원내각제적 요소는 무엇인가요?
답변: 1. 국무총리제, 2. 국회의원의 장관직 겸직 가능, 3. 행정부의 법률안 제출권입니다.

질문: 총재 정부란 무엇인가요?
답변: 총재 정부는 프랑스 혁명 당시 로베스피에르의 가혹한 공포 정치에 반발하여 일어난 정부로 5명의 총재가 행정을 담당하였습니다.



### 사용자 쿼리-데이터셋 답변 유사도 검색

In [16]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

import torch
import pandas as pd
import numpy as np

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS
from langchain.schema import Document

from kiwipiepy import Kiwi

file_path = '중학사회(취합)_카테고리 추가.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

# 질문과 답변 리스트 생성
questions = df['SENT_CONTENT'].tolist()
int_sentences = df['INT_SENTENCE'].tolist()
int_responses = df['INT_RESPONSE'].tolist()

# Kiwi 형태소 분석기 설정
kiwi = Kiwi()

# 형태소 분석을 통해 문서를 토큰화하는 함수
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 문서와 쿼리를 토큰화
tokenized_int_responses = [kiwi_tokenize(response) for response in int_responses]

# 사용자 질문에 대한 카테고리 결정
query = "우리나라 정부 형태가 뭐야?"
category = '정치'

# 특정 카테고리에 해당하는 문서만 필터링
filtered_documents = []
for i, response in enumerate(int_responses):
    doc_category = df.loc[i, 'CATEGORY']
    if doc_category == category:
        filtered_documents.append(Document(page_content=response, metadata={'index': i, 'category': doc_category}))

# BM25 모델 설정
kiwi_bm25 = BM25Retriever.from_documents(filtered_documents, preprocess_func=kiwi_tokenize)

# FAISS 설정
hf_model_name = "intfloat/multilingual-e5-large"  # contextualized embedding model 사용
hf_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name)
faiss = FAISS.from_documents(filtered_documents, hf_embeddings).as_retriever()

In [17]:
# EnsembleRetriever에서 검색 수행
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

# 검색 수행
results = retriever.invoke(query)

# 중복되지 않은 int_sentence를 저장할 set
unique_sentences = set()

# 상위 3개의 중복되지 않은 int_sentence를 찾는 리스트
top_results = []

# 결과에서 중복되지 않은 int_sentence만 저장
for result in results:
    index = result.metadata['index']  # 검색된 결과의 인덱스
    if int_sentences[index] not in unique_sentences:
        unique_sentences.add(int_sentences[index])  # 중복 제거
        top_results.append(result)
    if len(top_results) == 3:  # 3개의 고유한 int_sentence가 채워지면 종료
        break

# 결과 출력
print("사용자 질문:", query)
print("\n상위 3개 결과:")

for result in top_results:
    index = result.metadata['index']
    print(f"질문: {int_sentences[index]}")
    print(f"답변: {int_responses[index]}")
    print()

사용자 질문: 우리나라 정부 형태가 뭐야?

상위 3개 결과:
질문: 우리나라의 정부 형태에 대해 알려주세요.
답변: 우리나라의 정부 형태는 의원내각제적 요소를 가미한 대통령제 입니다.

질문: 대통령제와 의원 내각제의 차이점이 무엇인가요?
답변: 대통령제는 입법부와 행정부가 엄격하게 분리된 정부 형태입니다.
의원 내각제는 입법부와 행정부의 권한이 융합된 정부 형태입니다.
대통령제와 의원 내각제를 구분하는 방법은 정부와 의회의 융합 정도를 보면 됩니다.
정부와 의회가 철저하게 구분되어 있으면 대통령제, 정부와 의회가 융합되어 있으면 의원 내각제입니다.

질문: 양원제와 단원제가 무엇인가요?
답변: 양원제는 의회가 상원과 하원으로 나뉘어 각각 정무를 따로 맡는 국회 형태이고,
단원제는 의회가 단일화되어 모든 정무를 일괄로 맡는 국회 형태를 일컫습니다.
현재 우리나라는 상원과 하원으로 구분되지 않는 단원제입니다.

